In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset
import re

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    
    # Remove usernames and URLs
    text = re.sub(r'^@\S+\s*', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    text = text.lower()
        
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

def load_and_prepare_data():
    normal_df = pd.read_csv("normal_final.csv", skiprows=1, names=["text", "label"])
    hate_df = pd.read_csv("hate_final.csv", skiprows=1, names=["text", "label"])
    normal_df["label"] = 0
    hate_df["label"] = 1
    df = pd.concat([normal_df, hate_df], ignore_index=True)
    df["text"] = df["text"].apply(clean_text)
    print(df["label"].value_counts())
    return df

df = load_and_prepare_data()

# Split data
train_df, val_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42,
    stratify=df["label"] 
)

Using device: cuda
label
0    20000
1    20000
Name: count, dtype: int64


In [ ]:
# Load DziriBERT Tokenizer & Model
MODEL_NAME = "alger-ia/dziribert"
print(f"\nLoading model: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,  
    hidden_dropout_prob=0.3,
    attention_probs_dropout_prob=0.3,
    problem_type="single_label_classification"
).to(device)


Loading model: alger-ia/dziribert


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at alger-ia/dziribert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Create datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )


train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro"  
    )
    
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

# ✅ Step 6: Training configuration
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    logging_steps=100,
    seed=42,
    fp16=True,
    gradient_accumulation_steps=2
)

c:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=2)
    ]
)

print("\nStarting training...")
trainer.train()


Starting training...


TrainOutput(global_step=3000, training_loss=0.25765134716033933, metrics={'train_runtime': 1570.8358, 'train_samples_per_second': 122.228, 'train_steps_per_second': 1.91, 'total_flos': 1.262933065728e+16, 'train_loss': 0.25765134716033933, 'epoch': 6.0})

In [ ]:
# Save the best model
model.save_pretrained("./dziri_binary_final")
tokenizer.save_pretrained("./dziri_binary_final")
trainer.save_state()
print("\n✅ Model trained and saved!")


✅ Model trained and saved!


In [ ]:
# Evaluation
print("\nFinal Evaluation Results:")
eval_results = trainer.evaluate(val_dataset)
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")


Final Evaluation Results:


eval_loss: 0.4032
eval_accuracy: 0.8744
eval_f1: 0.8743
eval_precision: 0.8750
eval_recall: 0.8744
eval_runtime: 16.2200
eval_samples_per_second: 493.2190
eval_steps_per_second: 15.4130
epoch: 6.0000


In [ ]:
# inference
def predict_hate_speech(texts):
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**encodings)
    predictions = torch.argmax(outputs.logits, dim=1)
    return predictions.cpu().numpy()

example_texts = [
    "نكرهك ya lkhmaj",
    "أنت كافر يا عدو الله",
    "يا المروكي روح بوس يد الملك",
    "7na bakri kna 3andna l7a9",
    "أنت عار على الجزائر",
    "يحيا الجيش الوطني الشعبي",
    " 🥵 بوتين يخدم عند تبون",
    "تبون هو الرئيس",
    "الله لا تربحك يا سلال",
    "YA CHBAB ! YA L3AMI9 ! na7kmek ngat3ak",
    "الراس يطير صدقني الراس يطير يا جدك",
    "تجي معايا نتسابق في الحقول 🥹 ؟",
    "Nchllh fiha khir",
    "صحا رمضانكم يا جماعة",
    "ro7 akhtini allah yostrok",
    "نكره العرب",
    "امشي منا يا الكحلوش", 
    "العرب كلهم خونة",
    "انها زنجية من افريقيا اللعينة",
    "n7abek ya lbabay",
    "les kabyles mch rojala",
    "السعوديات رقاصات موسم الرياض",
    "تزوجي بيا 🥹 ؟",
    "نتا قصير و كحلوش, ما تقدرش تلعب فوتبول",
    "النساء مكانهن المطبخ",
]

predictions = predict_hate_speech(example_texts)

print("\nExample Predictions:")
for text, pred in zip(example_texts, predictions):
    print(f"Text: {text} ---→ Prediction: {'🚨 HATE SPEECH' if pred == 1 else '✅ Normal'}")

NameError: name 'tokenizer' is not defined